# VacationPy
----

In [1]:
# Dependencies and Setup:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key:
from api_keys import g_key

In [2]:
# Reading the 'csv file' with our weather data:
data = pd.read_csv('../OutputData/cities.csv')
data

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mount Isa,-20.73,139.50,48.20,57,0,8.05,AU,1594586492
1,Erzin,36.96,36.20,80.01,87,20,3.00,TR,1594586492
2,Khovd,48.01,91.64,58.82,74,35,0.72,MN,1594586382
3,Upata,8.01,-62.40,91.00,60,11,1.99,VE,1594586493
4,Busselton,-33.65,115.33,54.16,87,37,13.87,AU,1594586493
...,...,...,...,...,...,...,...,...,...
540,Zapolyarnyy,69.42,30.81,44.60,87,90,11.41,RU,1594586633
541,Dabat,12.98,37.77,53.69,100,100,5.48,ET,1594586633
542,Sinnamary,5.38,-52.95,79.81,86,48,10.07,GF,1594586514
543,Mporokoso,-9.37,30.12,63.09,50,0,7.47,ZM,1594586633


In [3]:
# Pulling the information I will need to populate my ideal weather conditions dataframe:
lat_long = data[['Latitude', 'Longitude']]
lat_long

,Latitude,Longitude
0,-20.73,139.50
1,36.96,36.20
2,48.01,91.64
3,8.01,-62.40
4,-33.65,115.33
...,...,...
540,69.42,30.81
541,12.98,37.77
542,5.38,-52.95
543,-9.37,30.12


In [4]:
# Pulling the 'humidity' to add to the heatmap:
humidity = data['Humidity']
humidity

0       57
1       87
2       74
3       60
4       87
      ... 
540     87
541    100
542     86
543     50
544     44
Name: Humidity, Length: 545, dtype: int64

In [5]:
# Generating the heatmap for the cities we have pulled:
gmaps.configure(api_key = g_key)
fig = gmaps.figure(zoom_level=2, center=(20,30))
heat_layer = gmaps.heatmap_layer(lat_long, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Setting weather conditions to pull the cities with the best weather to find the vacation spots:
# I specifically set the winds to be less than 5 so I can populate no more than 10 cities/places:
ideal_weather = data.loc[(data['Max Temperature'] > 70) & 
                                 (data['Max Temperature'] < 80) & 
                                 (data['Wind Speed'] < 5) & 
                                 (data['Cloudiness'] == 0)]
ideal_weather

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
120,Dawlatabad,36.41,64.91,74.80,35,0,2.15,AF,1594586523
170,Pompéia,-22.11,-50.17,72.16,57,0,3.69,BR,1594586537
183,Heraclea,37.51,34.05,70.05,33,0,1.95,TR,1594586540
247,Arrondissement de Mont-de-Marsan,44.00,-0.67,70.83,56,0,2.24,FR,1594586557
310,Berdigestyakh,62.10,126.70,73.90,81,0,2.28,RU,1594586573
329,Mastung,29.80,66.84,70.99,55,0,3.69,PK,1594586577
342,Formosa do Rio Preto,-11.05,-45.19,77.31,39,0,4.16,BR,1594586581
407,Mitsamiouli,-11.38,43.28,77.52,73,0,4.94,KM,1594586598


In [7]:
# Assuring there is 10 or less cities/places populating:
ideal_weather.count()

City               8
Latitude           8
Longitude          8
Max Temperature    8
Humidity           8
Cloudiness         8
Wind Speed         8
Country            8
Date               8
dtype: int64

In [8]:
# Using '.copy()' will help not interupt my existing 'ideal_weather' dataframe:
hotel_df = ideal_weather.copy()
hotel_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
120,Dawlatabad,36.41,64.91,74.80,35,0,2.15,AF,1594586523
170,Pompéia,-22.11,-50.17,72.16,57,0,3.69,BR,1594586537
183,Heraclea,37.51,34.05,70.05,33,0,1.95,TR,1594586540
247,Arrondissement de Mont-de-Marsan,44.00,-0.67,70.83,56,0,2.24,FR,1594586557
310,Berdigestyakh,62.10,126.70,73.90,81,0,2.28,RU,1594586573
329,Mastung,29.80,66.84,70.99,55,0,3.69,PK,1594586577
342,Formosa do Rio Preto,-11.05,-45.19,77.31,39,0,4.16,BR,1594586581
407,Mitsamiouli,-11.38,43.28,77.52,73,0,4.94,KM,1594586598


In [9]:
# Adding a new column for 'Hotel Name':
hotel_df['Hotel Name'] = ""
hotel_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
120,Dawlatabad,36.41,64.91,74.80,35,0,2.15,AF,1594586523,
170,Pompéia,-22.11,-50.17,72.16,57,0,3.69,BR,1594586537,
183,Heraclea,37.51,34.05,70.05,33,0,1.95,TR,1594586540,
247,Arrondissement de Mont-de-Marsan,44.00,-0.67,70.83,56,0,2.24,FR,1594586557,
310,Berdigestyakh,62.10,126.70,73.90,81,0,2.28,RU,1594586573,
329,Mastung,29.80,66.84,70.99,55,0,3.69,PK,1594586577,
342,Formosa do Rio Preto,-11.05,-45.19,77.31,39,0,4.16,BR,1594586581,
407,Mitsamiouli,-11.38,43.28,77.52,73,0,4.94,KM,1594586598,


In [10]:
# Searching for the closest hotels to the cities that meet the weather criteria:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {
    "key": g_key,
    "radius": 5000,
    "type": "lodging",
    "keyword": "hotel"
}

for index, row in hotel_df.iterrows():    
    ideal_lat = row['Latitude']
    ideal_long = row['Longitude']
    params['location'] = f"{ideal_lat}, {ideal_long}"
    response = requests.get(url, params=params).json()
    results = response['results']   
# pprint(results)  

    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "-None available-"
        
        pass      

In [11]:
# Printing my 'hotel_df' to display the information pulled from the API Calls:
hotel_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
120,Dawlatabad,36.41,64.91,74.80,35,0,2.15,AF,1594586523,-None available-
170,Pompéia,-22.11,-50.17,72.16,57,0,3.69,BR,1594586537,Hotel Pousada Vale do Sol
183,Heraclea,37.51,34.05,70.05,33,0,1.95,TR,1594586540,Mirel Hotel
247,Arrondissement de Mont-de-Marsan,44.00,-0.67,70.83,56,0,2.24,FR,1594586557,Le moulin vieux
310,Berdigestyakh,62.10,126.70,73.90,81,0,2.28,RU,1594586573,-None available-
329,Mastung,29.80,66.84,70.99,55,0,3.69,PK,1594586577,Nawab Hotel
342,Formosa do Rio Preto,-11.05,-45.19,77.31,39,0,4.16,BR,1594586581,Porto Center Hotel
407,Mitsamiouli,-11.38,43.28,77.52,73,0,4.94,KM,1594586598,Moina Lisa


In [12]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
# Add marker layer on top of heat map
# Display figure:

fig = gmaps.figure(zoom_level=1.5, center=(20,30))
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
heat_layer = gmaps.heatmap_layer(lat_long, weights=humidity, dissipating=False, max_intensity=100, point_radius=1.5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))